In [53]:
import os, os.path
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Input, Conv2D, MaxPooling2D, Flatten, Softmax, Conv1D, MaxPooling1D
from keras import optimizers, regularizers
from sklearn import metrics
import seaborn as sns
import keras
import time
import random
from hyperopt import fmin, tpe, hp
from pydub import AudioSegment 
import shutil

In [15]:
bgrnd_dir = 'speechData/_background_noise_'
wavfiles = [wavfile for wavfile in os.listdir(bgrnd_dir) if '.wav' in wavfile]
wavfiles = [os.path.join(bgrnd_dir, wavfile) for wavfile in wavfiles]

In [16]:
def background_noise_maker(input_path, dirname):
    audio = AudioSegment.from_wav(input_path)
    duration = len(audio)

    for i in range(0, duration, 1000):
        sample = audio[i:i+1000]
        filename = f'{dirname}/sample_{int(np.round(np.random.random(1)*10000000)[0])}.wav'
        sample.export(filename, format='wav')
        
shutil.rmtree('speechData/background')
        
os.mkdir('speechData/background')
_ = [background_noise_maker(wavfile, 'speechData/background') for wavfile in wavfiles]

shutil.rmtree('speechData\\_background_noise_')

In [69]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory='speechData',
    labels="inferred",
    batch_size=128,
    output_sequence_length=1000,
    seed=0,
    validation_split=0.2,
    subset='both',
)

Found 10402 files belonging to 11 classes.
Using 8322 files for training.
Using 2080 files for validation.


In [70]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

In [71]:
def popper(list):
    
    list.pop(-1)

    return list

x_train = [x for x, y in train_ds.take(-1)]
y_train = [y for x, y in train_ds.take(-1)]

x_val = [x for x, y in val_ds.take(-1)]
y_val = [y for x, y in val_ds.take(-1)]

x_train = popper(x_train)
y_train = popper(y_train)

x_val = popper(x_val)
y_val = popper(y_val)

In [80]:
# Initialise the model
model = Sequential()

# Input Layer
model.add(Input(shape=(128, 1000)))

# Convolutional Layers: nothing special here just skips adding a pooling layer at whatever the last convolutional layer is to accommdate for the dropout
for i in range(5):
    model.add(Conv1D(32, kernel_size = 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    if i != 5-1:
        model.add(MaxPooling1D(pool_size =1, strides=2, padding= 'same'))

model.add(Dropout((0.2)))
# Final Layer
model.add(Flatten())
model.add(Dense(11))
model.add(Softmax())

# Set the optimization options and compile the model
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_26 (Conv1D)          (None, 128, 32)           96032     
                                                                 
 batch_normalization_25 (Bat  (None, 128, 32)          128       
 chNormalization)                                                
                                                                 
 activation_25 (Activation)  (None, 128, 32)           0         
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 64, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_27 (Conv1D)          (None, 64, 32)            3104      
                                                                 
 batch_normalization_26 (Bat  (None, 64, 32)          

In [81]:
history = model.fit(x_train, y_train, batch_size=16, epochs=20, validation_split=0.2, verbose=1)

Epoch 1/20


ValueError: in user code:

    File "c:\ProgramData\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\ProgramData\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\ProgramData\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\ProgramData\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\ProgramData\anaconda3\envs\tf_gpu\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\ProgramData\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\input_spec.py", line 216, in assert_